<a href="https://colab.research.google.com/github/AmisheeChoksi/IE_Capstone_KPMG_Group9/blob/main/Capstone_Pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SALES PREDICTIONS FOR ACME TEXTILE CORP

This project was developed for the IE Capstone Project with the dataset provided by KPMG of ACME Textile Corp.

1. [Import Libraries](#section-1)
2. [Import Dataset](#section-2)
3. [Modeling](#section-3)
4. [Modeling II](#section-4)

## **1. Import Libraries**
<a id='section-1'></a>

In [ ]:
#Import libraries
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline  

import pandas as pd

import numpy as np
import itertools
from collections import defaultdict
from operator import itemgetter
import datetime as dt
#from pandas_profiling import ProfileReport
#%pip install pandas-profiling
#%pip install xlrd
#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

# Configure Pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
!pip install --force-reinstall numpy==1.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.0-cp38-cp38-manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.0 which is incompatible.
xarray-einstats 0.5.1 requires scipy>=1.6, but you have scipy 1.5.4 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.9 which is incompatible.
cmdstanpy 1.1.0 requires numpy>=1.21, but you have numpy 1.20.0 which is incompatible.


In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 16.8 MB/s eta 0:00:00
  Preparing metadat

## **2.Import Dataset**
<a id='section-2'></a>

All the data was taken from the dataset provided by KPMG. The dataset has the following columns:
- year-week: year and number of week inside a year
- ID_material: Internal identification number of the product
- Units: Units of sales for each product every week of the year

In [ ]:
url = 'https://raw.githubusercontent.com/AmisheeChoksi/IE_Capstone_KPMG_Group9/main/IE%20KPMG_Demand%20Forecast%20dataset.csv'
df = pd.read_csv(url)

In [ ]:
!mkdir trained_models

In [ ]:
df.head(5)

,year-week,ID_material,Units
0,2019-11,120010970,540
1,2019-12,120010970,750
2,2019-13,120010970,960
3,2019-14,120010970,720
4,2019-15,120010970,1620


In [ ]:
df=df.sort_values(['year-week','ID_material'])

In [ ]:
df.head()

,year-week,ID_material,Units
1381,2017-02,120004096,120
1382,2017-03,120004096,120
1383,2017-04,120004096,300
1384,2017-05,120004096,60
1385,2017-06,120004096,120


In [ ]:
# Create date column
df['date'] = pd.to_datetime(df['year-week'] + '-6', format='%G-%V-%u')

In [ ]:
df.head()

,year-week,ID_material,Units,date
1381,2017-02,120004096,120,2017-01-14
1382,2017-03,120004096,120,2017-01-21
1383,2017-04,120004096,300,2017-01-28
1384,2017-05,120004096,60,2017-02-04
1385,2017-06,120004096,120,2017-02-11


In [ ]:
df.tail()

,year-week,ID_material,Units,date
303,2022-39,120012606,1365,2022-10-01
2348,2022-39,120014486,756,2022-10-01
2616,2022-39,120014488,2160,2022-10-01
1242,2022-39,120015842,280,2022-10-01
2856,2022-39,120015996,276,2022-10-01


In [ ]:
# extract features from date**
df['month'] = [i.month for i in df['date']]
df['year'] = [i.year for i in df['date']]

In [ ]:
df.head()

,year-week,ID_material,Units,date,month,year
1381,2017-02,120004096,120,2017-01-14,1,2017
1382,2017-03,120004096,120,2017-01-21,1,2017
1383,2017-04,120004096,300,2017-01-28,1,2017
1384,2017-05,120004096,60,2017-02-04,2,2017
1385,2017-06,120004096,120,2017-02-11,2,2017


In [ ]:
#Cleaning the "Units" column by converting the column into a float first.
df['Units'] = pd.to_numeric(df['Units'], errors='coerce').fillna(0)

#Now that the column is a float we need to round up the values that have a decimal value and convert it to an integer value.
df['Units'] = df['Units'].apply(lambda x: int(np.ceil(x)))

In [ ]:
df['ID_material'] = df['ID_material'].astype('category')

In [ ]:
df = df.drop('year-week', axis=1)

In [ ]:
df.dtypes

ID_material          category
Units                   int64
date           datetime64[ns]
month                   int64
year                    int64
dtype: object

In [ ]:
df['ID_material'].nunique()

14

In [ ]:
# plot multiple time series with moving avgs in a loop

import plotly.express as px

for i in df['ID_material'].unique():
    subset = df[df['ID_material'] == i]
    subset['moving_average'] = subset['Units'].rolling(3).mean()
    fig = px.line(subset, x="date", y=["Units", "moving_average"], title = i, template = 'plotly_dark')
    fig.show()

<ipython-input-19-b35bda1b7475>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-19-b35bda1b7475>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## **3.Modeling**
<a id='section-3'></a>

Let's start the training process

In [ ]:
np.version.version

'1.22.4'

In [ ]:
df.tail()

,ID_material,Units,date,month,year
303,120012606,1365,2022-10-01,10,2022
2348,120014486,756,2022-10-01,10,2022
2616,120014488,2160,2022-10-01,10,2022
1242,120015842,280,2022-10-01,10,2022
2856,120015996,276,2022-10-01,10,2022


In [ ]:
import sys
!{sys.executable} -m pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pycaret.regression import *

In [ ]:
from tqdm import tqdm

In [ ]:
from tqdm import tqdm
from pycaret.regression import *

all_ts = df['ID_material'].unique()

all_results = []
final_model = {}

for i in tqdm(all_ts):
    
    df_subset = df[df['ID_material'] == i]
    
    # initialize setup from pycaret.regression
    s = setup(df_subset, target = 'Units', train_size = 0.9,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 4,
              ignore_features = ['date', 'ID_material'],
              numeric_features = ['month', 'year'],
              silent = True, verbose = False, session_id = 321)
    
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=False)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['ID_material'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # save transformation pipeline and model as pickle file 
    save_model(f, model_name='/content/trained_models/' + str(i), verbose=False)

Streaming output truncated to the last 5000 lines.
INFO:logs:create_model(estimator=lightgbm, fold=TimeSeriesSplit(max_train_size=None, n_splits=4), round=4, cross_validation=True, predict=True, fit_kwargs={}, groups=None, refit=False, verbose=False, system=False, metrics=None, experiment_custom_tags=None, add_to_model_list=True, probability_threshold=None, display=<pycaret.internal.Display.Display object at 0x7fd0d954afd0>, return_train_score=False, kwargs={})
INFO:logs:Checking exceptions
INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:Light Gradient Boosting Machine Imported succesfully
INFO:logs:Starting cross validation
INFO:logs:Cross validating with TimeSeriesSplit(max_train_size=None, n_splits=4), n_jobs=-1
INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Uploading results into container
INFO:logs:Uploading model into cont

In [ ]:
concat_results = pd.concat(all_results,axis=0)
concat_results.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),ID_material
ada,AdaBoost Regressor,73.9974,8.288671e+03,90.9661,-0.0658,2.1809,0.4760,0.0275,120004096
par,Passive Aggressive Regressor,1490.6467,3.580593e+06,1887.8715,-0.1874,2.5444,1.3073,0.0175,120014488
en,Elastic Net,1011.6320,1.556424e+06,1241.2169,-0.1754,1.2511,0.6781,0.0225,120014486
dummy,Dummy Regressor,479.3601,3.401380e+05,581.1534,-0.2029,2.6091,0.8968,0.0175,120009816
dummy,Dummy Regressor,707.9970,8.029836e+05,890.4510,-0.1205,0.8736,1.0351,0.0275,120015996


In [ ]:
df.tail()

,ID_material,Units,date,month,year
303,120012606,1365,2022-10-01,10,2022
2348,120014486,756,2022-10-01,10,2022
2616,120014488,2160,2022-10-01,10,2022
1242,120015842,280,2022-10-01,10,2022
2856,120015996,276,2022-10-01,10,2022


In [ ]:
# create a date range for the desired periods
all_dates = pd.date_range(start='2022-10-08', periods = 8, freq = 'W')
# create empty dataframe
score_df = pd.DataFrame()
# add columns to dataset
score_df['date'] = all_dates
score_df['month'] = [i.month for i in score_df['date']]
score_df['year'] = [i.year for i in score_df['date']]
score_df

,date,month,year
0,2022-10-09,10,2022
1,2022-10-16,10,2022
2,2022-10-23,10,2022
3,2022-10-30,10,2022
4,2022-11-06,11,2022
5,2022-11-13,11,2022
6,2022-11-20,11,2022
7,2022-11-27,11,2022


In [ ]:
from pycaret.regression import load_model, predict_model

all_score_df = []

for i in tqdm(df['ID_material'].unique()):
    l = load_model('/content/trained_models/' + str(i), verbose=False)
    p = predict_model(l, data=score_df)
    p['ID_material'] = i
    all_score_df.append(p)
concat_df = pd.concat(all_score_df, axis=0)
concat_df.head()

  0%|          | 0/14 [00:00<?, ?it/s]INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/trained_models/120004096, platform=None, authentication=None, verbose=False)
INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False,
                                      features_todrop=['date', 'ID_material'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['month', 'year'],
                                      target='Units', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_valu...
                ('dummy', Dummify(target='Units')),
 

,date,month,year,Label,ID_material
0,2022-10-09,10,2022,115.901639,120004096
1,2022-10-16,10,2022,115.901639,120004096
2,2022-10-23,10,2022,115.901639,120004096
3,2022-10-30,10,2022,115.901639,120004096
4,2022-11-06,11,2022,115.901639,120004096


In [ ]:
final_df = df.append(concat_df)

In [ ]:
final_df.sort_values('date', ascending=True)

,ID_material,Units,date,month,year,Label
1381,120004096,120.0,2017-01-14,1,2017,NaN
1382,120004096,120.0,2017-01-21,1,2017,NaN
1383,120004096,300.0,2017-01-28,1,2017,NaN
1384,120004096,60.0,2017-02-04,2,2017,NaN
1385,120004096,120.0,2017-02-11,2,2017,NaN
...,...,...,...,...,...,...
7,120010342,NaN,2022-11-27,11,2022,636.761204
7,120010970,NaN,2022-11-27,11,2022,1556.542073
7,120014486,NaN,2022-11-27,11,2022,2137.359375
7,120012154,NaN,2022-11-27,11,2022,862.899143


In [ ]:
final_df['ID_material'] = final_df['ID_material'].astype('str')

In [ ]:
for i in final_df['ID_material'].unique()[:]:
    sub_df = final_df[final_df['ID_material'] == i]
    
    import plotly.express as px
    fig = px.line(sub_df, x="date", y=['Units', 'Label'], title=i, template = 'plotly_dark')
    fig.show()

## **4. Modeling II**
<a id='section-4'></a>

In [ ]:
!mkdir trained_models2

In [ ]:
all_ts = df['ID_material'].unique()

all_results = []
final_model = {}

for i in tqdm(all_ts):
    
    df_subset = df[df['ID_material'] == i]
    
    # initialize setup from pycaret.regression
    s = setup(df_subset, target = 'Units', train_size = 0.85,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ['date', 'ID_material', 'month', 'year'],
              silent = True, verbose = False, session_id = 322)
    
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=False)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['ID_material'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # save transformation pipeline and model as pickle file 
    save_model(f, model_name='/content/trained_models2/' + str(i), verbose=False)

Streaming output truncated to the last 5000 lines.
INFO:logs:Uploading model into container now
INFO:logs:create_model_container: 21
INFO:logs:master_model_container: 21
INFO:logs:display_container: 2
INFO:logs:KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                    weights='uniform')
INFO:logs:create_model() succesfully completed......................................
                    metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                    weights='uniform') raised an exception or returned all 0.0, trying without fit_kwargs:
  File "/usr/local/lib/python3.8/dist-packages/pycaret/internal/tabular.py", line 2205, in compare_models
    assert np.sum(model_results.iloc[0]) != 0.0
AssertionError

INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=knn, fold=TimeSeriesSplit(max_train_size=None, n_splits=3), round=4, cross_validation=True, predict=Tru

In [ ]:
concat_results = pd.concat(all_results,axis=0)
concat_results.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),ID_material
lightgbm,Light Gradient Boosting Machine,73.1349,7.978862e+03,88.4944,-0.0534,2.1073,0.4360,0.0400,120004096
lightgbm,Light Gradient Boosting Machine,1673.2025,3.927625e+06,1971.7213,-0.3510,2.7143,1.3838,0.0233,120014488
lightgbm,Light Gradient Boosting Machine,999.8072,1.625312e+06,1260.6737,-0.2030,1.2568,0.5569,0.0333,120014486
lightgbm,Light Gradient Boosting Machine,481.3372,3.357641e+05,579.3249,-0.0610,2.7090,0.8961,0.0333,120009816
lightgbm,Light Gradient Boosting Machine,679.4578,7.511611e+05,864.0161,-0.1413,0.6841,1.0084,0.0333,120015996


In [ ]:
# create a date range for desired periods
all_dates = pd.date_range(start='2022-10-08', periods = 8, freq = 'W')
# create empty dataframe
score_df = pd.DataFrame()
# add columns to dataset
score_df['date'] = all_dates
score_df['month'] = [i.month for i in score_df['date']]
score_df['year'] = [i.year for i in score_df['date']]
score_df

,date,month,year
0,2022-10-09,10,2022
1,2022-10-16,10,2022
2,2022-10-23,10,2022
3,2022-10-30,10,2022
4,2022-11-06,11,2022
5,2022-11-13,11,2022
6,2022-11-20,11,2022
7,2022-11-27,11,2022


In [ ]:
all_score_df = []

for i in tqdm(df['ID_material'].unique()):
    l = load_model('/content/trained_models/' + str(i), verbose=False)
    p = predict_model(l, data=score_df)
    p['ID_material'] = i
    all_score_df.append(p)
concat_df = pd.concat(all_score_df, axis=0)
concat_df.head()

  0%|          | 0/14 [00:00<?, ?it/s]INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/trained_models/120004096, platform=None, authentication=None, verbose=False)
INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False,
                                      features_todrop=['date', 'ID_material'],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['month', 'year'],
                                      target='Units', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_valu...
                ('dummy', Dummify(target='Units')),
 

,date,month,year,Label,ID_material
0,2022-10-09,10,2022,115.901639,120004096
1,2022-10-16,10,2022,115.901639,120004096
2,2022-10-23,10,2022,115.901639,120004096
3,2022-10-30,10,2022,115.901639,120004096
4,2022-11-06,11,2022,115.901639,120004096


In [ ]:
final_df = df.append(concat_df)

In [ ]:
final_df.sort_values('date', ascending=True)

,ID_material,Units,date,month,year,Label
1381,120004096,120.0,2017-01-14,1,2017,NaN
1382,120004096,120.0,2017-01-21,1,2017,NaN
1383,120004096,300.0,2017-01-28,1,2017,NaN
1384,120004096,60.0,2017-02-04,2,2017,NaN
1385,120004096,120.0,2017-02-11,2,2017,NaN
...,...,...,...,...,...,...
7,120010342,NaN,2022-11-27,11,2022,636.761204
7,120010970,NaN,2022-11-27,11,2022,1556.542073
7,120014486,NaN,2022-11-27,11,2022,2137.359375
7,120012154,NaN,2022-11-27,11,2022,862.899143


In [ ]:
final_df['ID_material'] = final_df['ID_material'].astype('str')

In [ ]:
for i in final_df['ID_material'].unique()[:]:
    sub_df = final_df[final_df['ID_material'] == i]
    
    import plotly.express as px
    fig = px.line(sub_df, x="date", y=['Units', 'Label'], title=i, template = 'plotly_dark')
    fig.show()

We used the Pycaret library in order to perform parallel time series forcasting and automatically compare different models. The experiment has proven interesting, showing that decent results could be obtained by using LGBM modelling. However, the predictions did not look plausible, as most of them simply followed the average of the related time series.